In [5]:
import requests
import pandas as pd
from datetime import datetime

In [6]:
url = 'http://dataservice.accuweather.com/currentconditions/v1/topcities/150?apikey='
api_key = 'behGGPCV9pYyEfwrUqPfIjb4axaqJZ0a'

full_url = url + api_key

response = requests.get(full_url).json()

# Inicializamos listas para almacenar los datos obtenidos de la API

nombres_paises = []
nombres_ciudades = []
descripcion = []
temperatura = []
fecha_medicion = []
datetimes = []

# Iteramos sobre cada elemento del JSON obtenido
for data in response:
    # Extraemos los datos relevantes
    nombre_pais = data['Country']['LocalizedName']
    nombre_ciudad = data['LocalizedName']
    weather_text = data['WeatherText']
    temperature = data['Temperature']['Metric']['Value']
    timestamp = data['LocalObservationDateTime']
    datetime_insertion = datetime.now()

    # Agrega los datos a las listas+
    nombres_paises.append(nombre_pais)
    nombres_ciudades.append(nombre_ciudad)
    descripcion.append(weather_text)
    temperatura.append(temperature)
    fecha_medicion.append(timestamp)

    # Creamos DataFrame
    df = pd.DataFrame({
        "Nombre de País": nombres_paises,
        "Nombre de Ciudad": nombres_ciudades,
        "Descripción":descripcion,
        "Temperatura (C)": temperatura,
        "Fecha de Medición": fecha_medicion,
        "Fecha de Inserción": datetime_insertion
    })

df



,Nombre de País,Nombre de Ciudad,Descripción,Temperatura (C),Fecha de Medición,Fecha de Inserción
0,Bangladesh,Dhaka,Clouds and sun,22.8,2024-03-04T08:07:00+06:00,2024-03-03 23:15:47.916691
1,Democratic Republic of the Congo,Kinshasa,Some clouds,25.1,2024-03-04T02:56:00+01:00,2024-03-03 23:15:47.916691
2,Chile,Santiago,Clear,22.3,2024-03-03T22:55:00-03:00,2024-03-03 23:15:47.916691
3,China,Beijing,Cloudy,2.2,2024-03-04T09:48:00+08:00,2024-03-03 23:15:47.916691
4,Colombia,Bogota,Mostly cloudy,14.5,2024-03-03T20:55:00-05:00,2024-03-03 23:15:47.916691
...,...,...,...,...,...,...
145,Timor-Leste,Dili,Cloudy,31.1,2024-03-04T11:05:00+09:00,2024-03-03 23:15:47.916691
146,Indonesia,Banda Aceh,Cloudy,28.7,2024-03-04T09:05:00+07:00,2024-03-03 23:15:47.916691
147,Cyprus,Nicosia,Mostly clear,9.3,2024-03-04T04:07:00+02:00,2024-03-03 23:15:47.916691
148,Gibraltar,Gibraltar,Cloudy,15.0,2024-03-04T03:07:00+01:00,2024-03-03 23:15:47.916691


In [10]:
# Creación de conexión con RedShift
import psycopg2
url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
database = 'data-engineer-database'
user = 'marianolicera3_coderhouse'

with open('C:/Users/maria/Desktop/pwd_redshift.txt','r') as f:
    pwd = f.read()
try:
    conn = psycopg2.connect(
        host=url,
        dbname=database,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [11]:
#Crear la tabla si no existe

with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS marianolicera3_coderhouse.weather
        (
            nombre_pais VARCHAR(255),
            nombre_ciudad VARCHAR(255),
            descripcion VARCHAR(50),
            temperatura FLOAT,
            fecha_medicion DATETIME,
            fecha_insercion DATETIME 
        )
    """)
    conn.commit()
#nombre de país VARCHAR(200)
# nombre de ciudad VARCHAR(200)
# descripcion VARCHAR(50)
# temperatura FLOAT
# fecha de medicion DATETIME
# fecha de insercion DATETIME

In [12]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
    cur.execute("Truncate table weather")
    count = cur.rowcount

In [15]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO weather (nombre_pais, nombre_ciudad, descripcion, temperatura, fecha_medicion, fecha_insercion)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [16]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM weather")
results = cur.fetchall()
results

[('Bangladesh',
  'Dhaka',
  'Clouds and sun',
  22.8,
  datetime.datetime(2024, 3, 4, 8, 7),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('Democratic Republic of the Congo',
  'Kinshasa',
  'Some clouds',
  25.1,
  datetime.datetime(2024, 3, 4, 2, 56),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('Chile',
  'Santiago',
  'Clear',
  22.3,
  datetime.datetime(2024, 3, 3, 22, 55),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('China',
  'Beijing',
  'Cloudy',
  2.2,
  datetime.datetime(2024, 3, 4, 9, 48),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('Colombia',
  'Bogota',
  'Mostly cloudy',
  14.5,
  datetime.datetime(2024, 3, 3, 20, 55),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('Germany',
  'Berlin',
  'Mostly clear',
  8.4,
  datetime.datetime(2024, 3, 4, 3, 7),
  datetime.datetime(2024, 3, 3, 23, 15, 47, 916691)),
 ('Egypt',
  'Cairo',
  'Clear',
  15.0,
  datetime.datetime(2024, 3, 4, 3, 56),
  datetime.datetime(2024, 3, 3, 2